# データの読み込み

In [2]:
import pandas as pd
import numpy as np
import os

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test1/test.csv'
data_cleaned = pd.read_csv(data_path)

# データの先頭5行を表示
print("データの先頭5行:")
print(data_cleaned.head())

# データ型の確認
print("\nデータ型:")
print(data_cleaned.dtypes)

# 各列のユニークな値を表示
for column in data_cleaned.columns:
    unique_values = data_cleaned[column].unique()
    print(f"\n列 {column} のユニークな値 ({len(unique_values)}):\n", unique_values)

# データの行数と列数をプリントする
print(f"\nデータの行数: {data_cleaned.shape[0]}")
print(f"データの列数: {data_cleaned.shape[1]}")



データの先頭5行:
        ID  市区町村コード 最寄駅：距離（分）   間取り  面積（㎡）    建築年 建物の構造  建ぺい率（％）  容積率（％）  \
0  1000000     1101        26  ３ＬＤＫ     75  昭和64年    ＲＣ     40.0    60.0   
1  1000056     1101         1  ２ＬＤＫ     55  平成28年    ＲＣ     80.0   600.0   
2  1000108     1101         2    １Ｒ     15  昭和64年   ＳＲＣ     80.0   400.0   
3  1000109     1101         2  １ＬＤＫ     45   平成3年   ＳＲＣ     80.0   400.0   
4  1000110     1101         3    １Ｒ     20  昭和56年    ＲＣ     80.0   400.0   

         取引時点  
0  2020年第２四半期  
1  2020年第１四半期  
2  2020年第２四半期  
3  2020年第２四半期  
4  2020年第２四半期  

データ型:
ID             int64
市区町村コード        int64
最寄駅：距離（分）     object
間取り           object
面積（㎡）          int64
建築年           object
建物の構造         object
建ぺい率（％）      float64
容積率（％）       float64
取引時点          object
dtype: object

列 ID のユニークな値 (19466):
 [ 1000000  1000056  1000108 ... 47003880 47006648 47007421]

列 市区町村コード のユニークな値 (526):
 [ 1101  1102  1103  1104  1105  1106  1107  1108  1109  1110  1202  1203
  1204  1206  1207  12

# 最寄りの駅のデータの変換

In [3]:
import pandas as pd
import numpy as np
import os

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test1/test.csv'
data_cleaned = pd.read_csv(data_path)

#ステップ1: 最寄駅の特定の文字列を数値に変換する関数を定義
def convert_distance_to_minutes(distance):
    if isinstance(distance, str):
        if '30分?60分' in distance:
            return 45
        elif '1H?1H30' in distance:
            return 75
        elif '2H?' in distance:
            return 120
        elif '1H30?2H' in distance:
            return 105
        elif '分' in distance:
            return int(distance.replace('分', '').replace('?', ''))
        elif 'H' in distance:
            hours, minutes = 0, 0
            if 'H?' in distance:
                hours = int(distance.split('H')[0])
                return hours * 60
            elif 'H' in distance:
                hours, minutes = distance.split('H')
                minutes = minutes.replace('?', '')
                total_minutes = int(hours) * 60 + int(minutes) if minutes else int(hours) * 60
                return total_minutes
    return distance

# '最寄駅：距離（分）'列を変換
data_cleaned['最寄駅：距離（分）'] = data_cleaned['最寄駅：距離（分）'].apply(convert_distance_to_minutes)

# 数値データに変換
data_cleaned['最寄駅：距離（分）'] = pd.to_numeric(data_cleaned['最寄駅：距離（分）'], errors='coerce')

# 変換結果の確認
print(data_cleaned['最寄駅：距離（分）'].dtypes)
print(data_cleaned['最寄駅：距離（分）'].unique())

# データの行数と列数をプリントする
print(f"\nデータの行数: {data_cleaned.shape[0]}")
print(f"データの列数: {data_cleaned.shape[1]}")


int64
[ 26   1   2   3   0   4   6   5   7  11   8  12   9  13  10  45  15  14
  19  18  21  23  24  22  28  25  29  17  20  16 120  75 105  27]

データの行数: 19466
データの列数: 10


# 建築年のデータの変換

In [4]:
import pandas as pd
import numpy as np
import re

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test1/test.csv'
data_cleaned = pd.read_csv(data_path)

# ステップ 1: '平成', '昭和', '大正', '明治'を含まない、または欠損値を処理
def is_invalid_year_format(value):
    # '平成', '昭和', '大正', '明治' を含まない値、または欠損値は無効とみなす
    if pd.isna(value):
        return True
    if any(era in value for era in ['平成', '昭和', '大正', '明治']):
        return False
    return True

# 無効な値をNaNに変換
data_cleaned.loc[data_cleaned['建築年'].apply(is_invalid_year_format), '建築年'] = np.nan

# ステップ 2: 年号を西暦に変換する関数を定義
def convert_japanese_era_to_ad(year):
    if isinstance(year, str):
        if '平成' in year:
            return int(year.replace('平成', '').replace('年', '')) + 1988
        elif '昭和' in year:
            return int(year.replace('昭和', '').replace('年', '')) + 1925
        elif '大正' in year:
            return int(year.replace('大正', '').replace('年', '')) + 1911
        elif '明治' in year:
            return int(year.replace('明治', '').replace('年', '')) + 1867
    return np.nan

# '建築年'列を変換
data_cleaned['建築年'] = data_cleaned['建築年'].apply(convert_japanese_era_to_ad)

# ステップ 3: 数値データに変換
data_cleaned['建築年'] = pd.to_numeric(data_cleaned['建築年'], errors='coerce')

# ステップ 4: 1956年以前および2020年以降のデータはNaNに設定
data_cleaned.loc[(data_cleaned['建築年'] < 1956) | (data_cleaned['建築年'] > 2020), '建築年'] = np.nan

# ステップ 5: 欠損値を最頻値で補完
data_cleaned['建築年'].fillna(data_cleaned['建築年'].mode()[0], inplace=True)

# 変換結果の確認
print(data_cleaned['建築年'].dtypes)
print(data_cleaned['建築年'].unique())

# データの行数と列数をプリントする
print(f"\nデータの行数: {data_cleaned.shape[0]}")
print(f"データの列数: {data_cleaned.shape[1]}")


float64
[1989. 2016. 1991. 1981. 1984. 2011. 1977. 1985. 1986. 2015. 2007. 2013.
 1974. 2003. 1990. 2008. 2009. 2012. 2001. 2002. 1987. 1999. 2000. 1994.
 2006. 2018. 1996. 2004. 1979. 1988. 1998. 1975. 1976. 1970. 2005. 2014.
 1982. 2019. 1971. 1980. 1997. 1973. 1995. 1993. 1978. 1992. 1983. 2010.
 2017. 1972. 1968. 1964. 1967. 1969. 1961. 1965. 1966.]

データの行数: 19466
データの列数: 10


# 取引時点のデータの変換

In [5]:
import pandas as pd
import numpy as np
import re

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test1/test.csv'
data_cleaned = pd.read_csv(data_path)

# 確実に「取引時点」列が文字列型であることを確認
data_cleaned['取引時点'] = data_cleaned['取引時点'].astype(str)

# 取引時点を西暦に変換する関数
def convert_quarter_to_date(quarter_str):
    try:
        year, q = quarter_str.split('年第')
        year = int(year)
        if q == '１四半期':
            month = 3
        elif q == '２四半期':
            month = 6
        elif q == '３四半期':
            month = 9
        elif q == '４四半期':
            month = 12
        else:
            raise ValueError(f"Unexpected quarter format: {quarter_str}")
        return year * 100 + month
    except ValueError as e:
        print(f"Error processing {quarter_str}: {e}")
        return None

# 無効な取引時点をNaNに設定
data_cleaned['取引時点'] = data_cleaned['取引時点'].apply(lambda x: x if re.match(r'\d+年第[１-４]四半期', x) else np.nan)

# 取引時点を変換し、int64に変換
data_cleaned['取引時点'] = data_cleaned['取引時点'].apply(convert_quarter_to_date)

# 欠損値を最頻値で補完
data_cleaned['取引時点'].fillna(data_cleaned['取引時点'].mode()[0], inplace=True)

# 数値データに変換
data_cleaned['取引時点'] = pd.to_numeric(data_cleaned['取引時点'], errors='coerce')

# 変換結果の確認
print(data_cleaned['取引時点'].dtypes)
print(data_cleaned['取引時点'].unique())

# データの行数と列数をプリントする
print(f"\nデータの行数: {data_cleaned.shape[0]}")
print(f"データの列数: {data_cleaned.shape[1]}")


int64
[202006 202003]

データの行数: 19466
データの列数: 10


# 上記の処理を一括で行い、ファイルをtest2に保存する

In [6]:
import pandas as pd
import numpy as np
import os
import re

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test1/test.csv'
data_cleaned = pd.read_csv(data_path)

#ステップ 1: 最寄駅にある特定の文字列を数値に変換する関数を定義
def convert_distance_to_minutes(distance):
    if isinstance(distance, str):
        if '30分?60分' in distance:
            return 45
        elif '1H?1H30' in distance:
            return 75
        elif '2H?' in distance:
            return 120
        elif '1H30?2H' in distance:
            return 105
        elif '分' in distance:
            return int(distance.replace('分', '').replace('?', ''))
        elif 'H' in distance:
            hours, minutes = 0, 0
            if 'H?' in distance:
                hours = int(distance.split('H')[0])
                return hours * 60
            elif 'H' in distance:
                hours, minutes = distance.split('H')
                minutes = minutes.replace('?', '')
                total_minutes = int(hours) * 60 + int(minutes) if minutes else int(hours) * 60
                return total_minutes
    return distance

# '最寄駅：距離（分）'列を変換
data_cleaned['最寄駅：距離（分）'] = data_cleaned['最寄駅：距離（分）'].apply(convert_distance_to_minutes)

# 数値データに変換
data_cleaned['最寄駅：距離（分）'] = pd.to_numeric(data_cleaned['最寄駅：距離（分）'], errors='coerce')

# 変換結果の確認
print(data_cleaned['最寄駅：距離（分）'].dtypes)
print(data_cleaned['最寄駅：距離（分）'].unique())

# ステップ 2: 建築年の '平成', '昭和', '大正', '明治'を含まない、または欠損値を処理
def is_invalid_year_format(value):
    # '平成', '昭和', '大正', '明治' を含まない値、または欠損値は無効とみなす
    if pd.isna(value):
        return True
    if any(era in value for era in ['平成', '昭和', '大正', '明治']):
        return False
    return True

# 無効な値をNaNに変換
data_cleaned.loc[data_cleaned['建築年'].apply(is_invalid_year_format), '建築年'] = np.nan

# 年号を西暦に変換する関数を定義
def convert_japanese_era_to_ad(year):
    if isinstance(year, str):
        if '平成' in year:
            return int(year.replace('平成', '').replace('年', '')) + 1988
        elif '昭和' in year:
            return int(year.replace('昭和', '').replace('年', '')) + 1925
        elif '大正' in year:
            return int(year.replace('大正', '').replace('年', '')) + 1911
        elif '明治' in year:
            return int(year.replace('明治', '').replace('年', '')) + 1867
    return np.nan

# '建築年'列を変換
data_cleaned['建築年'] = data_cleaned['建築年'].apply(convert_japanese_era_to_ad)

# 数値データに変換
data_cleaned['建築年'] = pd.to_numeric(data_cleaned['建築年'], errors='coerce')

# 1956年以前および2020年以降のデータはNaNに設定
data_cleaned.loc[(data_cleaned['建築年'] < 1956) | (data_cleaned['建築年'] > 2020), '建築年'] = np.nan

# 欠損値を最頻値で補完
data_cleaned['建築年'].fillna(data_cleaned['建築年'].mode()[0], inplace=True)

# 変換結果の確認
print(data_cleaned['建築年'].dtypes)
print(data_cleaned['建築年'].unique())


# ステップ3: 取引時点の処理：　確実に「取引時点」列が文字列型であることを確認
data_cleaned['取引時点'] = data_cleaned['取引時点'].astype(str)

# 取引時点を西暦に変換する関数
def convert_quarter_to_date(quarter_str):
    try:
        year, q = quarter_str.split('年第')
        year = int(year)
        if q == '１四半期':
            month = 3
        elif q == '２四半期':
            month = 6
        elif q == '３四半期':
            month = 9
        elif q == '４四半期':
            month = 12
        else:
            raise ValueError(f"Unexpected quarter format: {quarter_str}")
        return year * 100 + month
    except ValueError as e:
        print(f"Error processing {quarter_str}: {e}")
        return None

# 無効な取引時点をNaNに設定
data_cleaned['取引時点'] = data_cleaned['取引時点'].apply(lambda x: x if re.match(r'\d+年第[１-４]四半期', x) else np.nan)

# 取引時点を変換し、int64に変換
data_cleaned['取引時点'] = data_cleaned['取引時点'].apply(convert_quarter_to_date)

# 欠損値を最頻値で補完
data_cleaned['取引時点'].fillna(data_cleaned['取引時点'].mode()[0], inplace=True)

# 数値データに変換
data_cleaned['取引時点'] = pd.to_numeric(data_cleaned['取引時点'], errors='coerce')

# 変換結果の確認
print(data_cleaned['取引時点'].dtypes)
print(data_cleaned['取引時点'].unique())


# データの保存
data_cleaned_path = '/Users/hayakawakazue/Downloads/data/test2/test.csv'
data_cleaned.to_csv(data_cleaned_path, index=False)

print(data_cleaned.dtypes)

# データの行数と列数を確認
print(f"保存されたデータの行数: {data_cleaned.shape[0]}")
print(f"保存されたデータの列数: {data_cleaned.shape[1]}")

# 保存されたデータを確認
print(f"データを保存しました: {data_cleaned_path}")

int64
[ 26   1   2   3   0   4   6   5   7  11   8  12   9  13  10  45  15  14
  19  18  21  23  24  22  28  25  29  17  20  16 120  75 105  27]
float64
[1989. 2016. 1991. 1981. 1984. 2011. 1977. 1985. 1986. 2015. 2007. 2013.
 1974. 2003. 1990. 2008. 2009. 2012. 2001. 2002. 1987. 1999. 2000. 1994.
 2006. 2018. 1996. 2004. 1979. 1988. 1998. 1975. 1976. 1970. 2005. 2014.
 1982. 2019. 1971. 1980. 1997. 1973. 1995. 1993. 1978. 1992. 1983. 2010.
 2017. 1972. 1968. 1964. 1967. 1969. 1961. 1965. 1966.]
int64
[202006 202003]
ID             int64
市区町村コード        int64
最寄駅：距離（分）      int64
間取り           object
面積（㎡）          int64
建築年          float64
建物の構造         object
建ぺい率（％）      float64
容積率（％）       float64
取引時点           int64
dtype: object
保存されたデータの行数: 19466
保存されたデータの列数: 10
データを保存しました: /Users/hayakawakazue/Downloads/data/test2/test.csv
